## Contexto do Projeto

Rossmann opera mais de 3.000 drogarias em 7 países europeus. Atualmente, os gerentes de loja da Rossmann têm a tarefa de prever suas vendas diárias com até seis semanas de antecedência. As vendas da loja são influenciadas por muitos fatores, incluindo promoções, competição, feriados escolares e estaduais, sazonalidade e localidade. Com milhares de gerentes individuais prevendo vendas com base em suas circunstâncias únicas, a precisão dos resultados pode ser bastante variada. Você pode baixar o conjunto de dados aqui: https://www.kaggle.com/c/rossmann-store-sales/data.

Problema de Negócio
Em primeiro lugar, precisamos entender qual é o nosso problema de negócios. Portanto, criamos um contexto para nos ajudar a construir a solução. Então, vamos seguir essas quatro etapas.

Qual é o contexto?
Em reunião com os líderes de cada departamento, o CEO da Rossmann fez a proposta de reformar todas as lojas.

Qual é a causa?
O CEO da Rossmann quer prever quanto cada loja vai vender nas próximas 6 semanas. Ele precisa saber se o orçamento será suficiente para fazer uma reforma em cada loja.

Quem vai liderar o projeto?
Precisamos de alguém que realmente saiba qual é o problema do negócio, porque ele vai liderar a solução. Portanto, ele é nosso stakeholder.

Como ficará nossa solução?

Qual é o formato?

Granularidade (hora, dia, produto) ---> 6 semanas

Tipo de problema (classificação, regressão, agrupamento, etc.) ---> Regressão

Como vamos entregar? (painel, csv, Notebook) ---> Notebook

Etapas do CRISP : QUESTAO DO NEGOCIO -> ENTENDIMENTO DO NEGOCIO -> COLETA DOS DADOS -> LIMPEZA DOS DADOS -> EXPLORACAO DOS DADOS -> MODELAGEM DOS DADOS -> ALGORITMOS DE MACHINE LEARNING

Etapas do Projeto:


Seguiremos todas essas etapas a seguir para a resolução do nosso projeto:


0.0. IMPORTS

1.0. DESCRIÇÃO DOS DADOS

2.0. FEATURE ENGINEERING

3.0. FILTRANDO OS DADOS

4.0. ANÁLISE EXPLORATÓRIA DOS DADOS

5.0. PREPARAÇÃO DOS DADOS

6.0. SELEÇÃO DE FEATURES

7.0. MACHINE LEARNING

8.0. INTERPRETANDO OS RESULTADOS

9.0. TRADUÇAO E INTERPRETACAO DO ERRO


# 0.0 Imports 

In [1]:
import math
import random
import datetime
import warnings
import inflection
import numpy   as np
import pandas  as pd
import seaborn as sns
import xgboost as xgb


import matplotlib.pyplot   as plt
from matplotlib.gridspec     import GridSpec

from scipy                   import stats    as ss
from boruta                import BorutaPy
from tabulate              import tabulate
from matplotlib            import pyplot as plt
from scipy.stats           import chi2_contingency
from IPython.display       import Image
from sklearn.metrics       import mean_absolute_error, mean_squared_error
from sklearn.ensemble      import RandomForestRegressor
from sklearn.linear_model  import LinearRegression, Lasso
from IPython.core.display  import HTML
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
import xgboost               as xgb
import pickle
from flask                   import Flask
from IPython.display         import Image

plt.rcParams['figure.figsize'] = [25, 12]
plt.rcParams.update({'font.size': 10})

warnings.filterwarnings('ignore')

### Loading data

In [2]:
df_sales_raw= pd.read_csv('train.csv',low_memory= False)
df_store_raw= pd.read_csv('store.csv',low_memory= False)

# Fazendo o merge dos dados para transformar os dois datasets em apenas 1 
df_raw=pd.merge(df_sales_raw,df_store_raw,how='left', on='Store')


In [3]:
df_raw.sample()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
592693,299,4,2014-01-16,4108,444,1,0,0,0,d,c,38630.0,9.0,2012.0,0,NaN,NaN,NaN


## Descrição dos dados das variáveis

Store - um ID único para cada loja.

DayOfWeek - Dia correspondente da semana.

Date - O dia do acontecimento do dado.

Sales - o volume de vendas.

Customers - o número de clientes em um determinado dia.

Open - um indicador para saber se a loja estava aberta: 0 = fechada, 1 = aberta

StateHoliday - indica um feriado estadual. Normalmente todas as lojas, com poucas exceções, fecham nos feriados estaduais. Observe que todas as escolas fecham nos feriados e finais de semana. a = feriado, b = feriado da Páscoa, c = Natal, 0 = Nenhum

SchoolHoliday - indica se (loja, data) foi afetado pelo fechamento de escolas públicas.

StoreType - diferencia entre 4 modelos de loja diferentes: a, b, c, d

Assortment - descreve um nível de sortimento: a = básico, b = extra, c = estendido

CompetitionDistance - distância em metros até a loja concorrente mais próxima.

CompetitionOpenSince [Month / Year] - fornece o mês e ano aproximados em que o concorrente mais próximo foi aberto.

Promo - indica se uma loja está fazendo uma promoção naquele dia.

Promo2 - Promo2 é uma promoção contínua e consecutiva para algumas lojas: 0 = a loja não está participando, 1 = a loja está participando.

Promo2Since [Year / Week] - descreve o ano e a semana em que a loja começou a participar da Promo2.

PromoInterval - descreve os intervalos consecutivos em que a Promo2 é iniciada, nomeando os meses em que a promoção é reiniciada. Por exemplo. "Fev, maio, agosto, novembro" significa que cada rodada começa em fevereiro, maio, agosto, novembro de qualquer ano para aquela loja.


## 0.1 Helper Functions

In [4]:
pd.set_option('max_columns', None)

def cramer_v(x, y):
    cm= np.asmatrix(pd.crosstab(x,y))
    n= cm.sum()
    r, k= cm.shape
    
    chi2= ss.chi2_contingency(cm)[0]
    chi2corr= max(0, chi2 - (k - 1) * (r-1)/n - 1)
    kcorr= k - (k - 1) ** 2 / (n - 1)
    rcorr= r - (r - 1) ** 2 / (n - 1)
    
    return np.sqrt((chi2corr / n) / (min(kcorr - 1, rcorr - 1)))

In [5]:
def cross_validation( x_training, kfold, model_name, model, verbose=False ):
    mae_list = []
    mape_list = []
    rmse_list = []
    for k in reversed( range( 1, kfold+1 ) ):
        if verbose:
            print( '\nKFold Number: {}'.format( k ) )
        # start and end date for validation 
        validation_start_date = x_training['date'].max() - datetime.timedelta( days=k*6*7)
        validation_end_date = x_training['date'].max() - datetime.timedelta( days=(k-1)*6*7)

        # filtering dataset
        training = x_training[x_training['date'] < validation_start_date]
        validation = x_training[(x_training['date'] >= validation_start_date) & (x_training['date'] <= validation_end_date)]

        # training and validation dataset
        # training
        xtraining = training.drop( ['date', 'sales'], axis=1 ) 
        ytraining = training['sales']

        # validation
        xvalidation = validation.drop( ['date', 'sales'], axis=1 )
        yvalidation = validation['sales']

        # model
        m = model.fit( xtraining, ytraining )

        # prediction
        yhat = m.predict( xvalidation )

        # performance
        m_result = ml_error( model_name, np.expm1( yvalidation ), np.expm1( yhat ) )

        # store performance of each kfold iteration
        mae_list.append(  m_result['MAE'] )
        mape_list.append( m_result['MAPE'] )
        rmse_list.append( m_result['RMSE'] )

    return pd.DataFrame( {'Model Name': model_name,
                          'MAE CV': np.round( np.mean( mae_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mae_list ), 2 ).astype( str ),
                          'MAPE CV': np.round( np.mean( mape_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( mape_list ), 2 ).astype( str ),
                          'RMSE CV': np.round( np.mean( rmse_list ), 2 ).astype( str ) + ' +/- ' + np.round( np.std( rmse_list ), 2 ).astype( str ) }, index=[0] )


def mean_percentage_error( y, yhat ):
    return np.mean( ( y - yhat ) / y )
     

    def mean_absolute_percentage_error( y, yhat ):
    return np.mean( np.abs( ( y - yhat ) / y ) ) #np.abs: retira o valor de resultado negativo

# funcao para uso em machine learning   
def ml_error( model_name, y, yhat ):
    mae = mean_absolute_error( y, yhat )
    mape = mean_absolute_percentage_error( y, yhat )
    rmse = np.sqrt( mean_squared_error( y, yhat ) )
    
    return pd.DataFrame( { 'Model Name': model_name, 
                           'MAE': mae, 
                           'MAPE': mape,
                           'RMSE': rmse }, index=[0] ) 
# df para armazenar os erros
# y: valor real, yhat : predito

IndentationError: expected an indented block (<ipython-input-5-81c2eb570571>, line 50)

In [ ]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    sns.set()

# 1.0 DATA DESCRIPTION


### 1.1 Rename Columns


In [ ]:
df1=df_raw.copy()  #criando uma copia do data para seguranca dos dados originais


In [ ]:
df1.columns      #olhando os nomes das colunas

In [ ]:
cols_old=['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo',
       'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment',
       'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'PromoInterval']
snakecase=lambda x:inflection.underscore(x) 
cols_new=list(map(snakecase,cols_old))  #lambda: e a declaracao de uma funcao 
#snakecase: reencreve em minusculo e com _ como separador
#funcao map: faz o mapeamento da funcao snakecase em todas as palavras da lista cols_old

#Rename
df1.columns=cols_new


### 1.2 Data Dimensions

In [ ]:
print('Numbers of Rows:{}'.format(df1.shape[0])) 
print('Numbers of Columns:{}'.format(df1.shape[1]))

### 1.3. Data Types

In [ ]:
df1.dtypes

In [ ]:
df1['date']=pd.to_datetime(df1['date'])  # ajustando a variavel de data 



### 1.4 Check NA

In [ ]:
df1.isna().sum()

### 1.5 Fillout NA

* competition_distance: Colocar uma distância muito grande para mostrar que não existe competição perto.
* competition_open_since_month: Colocar o respectivo número do mês da coluna ['date'].
* competition_open_since_year: Colocar o respectivo número do ano da coluna ['date'].
* promo2_since_week: Colocar o respectivo número da semana da coluna ['date'].
* promo2_since_year: Colocar o respectivo número do ano da coluna ['date'].
* promo_interval: Preencher com 0 e criar uma nova coluna chamada ['is_promo'] onde, se o mês da coluna['date'] estiver *na coluna ['promo_interval'] retorna 1(possui promoção) e 0 para o restante.

In [ ]:

#competition_distance (Na's faltantes:  2642 )

df1['competition_distance']=df1['competition_distance'].apply(lambda x:2000000.0 if math.isnan(x) else x)
# distancia em metros do competidor mais proximo . 
# math.isnam : encontra na coluna os valores sem dados 



In [ ]:
df1['competition_distance'].max()

In [ ]:
df1.isna().sum()

In [ ]:
df1.sample()

In [ ]:

#nao possui competidor proximo ou nao tem o dado
#No entendimento assumo que : se tiver dados vazios(isnan.math) na coluna 'competition since mounth/year/week', eu vou pegar a coluna 'date' extrair o mes e substituindo o NA, aplicando(apply)em todas as colunas (axis 1)
    
#competition_open_since_month (Na's faltantes: 323348)
#mes e ano que o competidor abriu 
df1['competition_open_since_month']=df1.apply(lambda x:x ['date'].month if math.isnan (x['competition_open_since_month'])else x ['competition_open_since_month'],axis=1)

#competition_open_since_year   ( Na's faltantes:  323348)
df1['competition_open_since_year']=df1.apply(lambda x:x ['date'].year if math.isnan (x['competition_open_since_year'])else x ['competition_open_since_year'],axis=1)

#promo2_since_week             ( Na's faltantes:  508031)
df1['promo2_since_week']=df1.apply(lambda x:x ['date'].week if math.isnan (x['promo2_since_week'])else x ['promo2_since_week'],axis=1)

#promo2_since_year             ( Na's faltantes:  508031)
df1['promo2_since_year']=df1.apply(lambda x:x ['date'].year if math.isnan (x['promo2_since_year'])else x ['promo2_since_year'],axis=1)

#promo_interval                  508031
#crio uma lista de meses, se a data estiver dentro da lista a promo2 foi ativa.

month_map={1:'Jan',2:'Fev',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
df1['promo_interval'].fillna(0,inplace=True) #fillna na coluna promo_interval com valor 0, #inplace=true para nao retornar nada, com modificacao direto na coluna 

df1['month_map']=df1['date'].dt.month.map(month_map) # substituindo os dados da coluna date para os meses da lista month map
df1['is_promo']=df1[['promo_interval','month_map']].apply(lambda x:0 if x['promo_interval']==0 else 1if x['month_map']in x['promo_interval'].split(',')else 0, axis=1)                                      



In [ ]:
df1.isna().sum()

In [ ]:
df1.sample(5).T

### 1.6 Change types


In [ ]:
df1.dtypes

In [ ]:
df1['competition_open_since_year']=df1['competition_open_since_year'].astype(int)
df1['competition_open_since_month']=df1['competition_open_since_month'].astype(int)
df1['promo2_since_week']=df1['promo2_since_week'].astype(int)
df1['promo2_since_year']=df1['promo2_since_year'].astype(int)

### 1.7 Descriptive Statistical


In [ ]:
# separando as variaveis numericas e as categoricas 
num_attributes= df1.select_dtypes(include=['int64','float64'])
cat_attributes= df1.select_dtypes(exclude=['int64','float64','datetime64[ns]'])

In [ ]:
num_attributes.sample(2)


In [ ]:
cat_attributes.sample(2)

### 1.7.1 Numerical Attributes

In [ ]:
# central tendency- mean, median
ct1=pd.DataFrame(num_attributes.apply(np.mean)).T
ct2=pd.DataFrame(num_attributes.apply(np.median)).T
#Dispersion- std,min,max,range,skew,kurtosis
d1=pd.DataFrame(num_attributes.apply(np.std)).T
d2=pd.DataFrame(num_attributes.apply(min)).T
d3=pd.DataFrame(num_attributes.apply(max)).T
d4=pd.DataFrame(num_attributes.apply(lambda x:x.max()-x.min())).T
d5=pd.DataFrame(num_attributes.apply(lambda x:x.skew())).T
d6=pd.DataFrame(num_attributes.apply(lambda x:x.kurtosis())).T
#concatenate
m=pd.concat([d2,d3,d4,ct1,ct2,d1,d5,d6]).T.reset_index()
m.columns=(['attributes','min','max','range','mean','median','std','skew','kurtosis'])


In [ ]:
m

In [ ]:
sns.distplot(df1['competition_distance']);


### 1.7.2  Categorical Attributes

In [ ]:
cat_attributes.apply(lambda x:x.unique().shape[0] )

In [ ]:
plt.figure(figsize= (17, 7))

aux1=df1[(df1['state_holiday']!='0')& (df1['sales']>0)];

plt.subplot(1,3,1)
sns.boxplot(x='state_holiday',y='sales', data= aux1);

plt.subplot(1,3,2)
sns.boxplot(x='store_type',y='sales',data=aux1);

plt.subplot(1,3,3)
sns.boxplot(x='assortment',y='sales',data=aux1);
plt.tight_layout()


# 2.0 Feature engineering

### 2.1 Mapa das hipóteses 

In [ ]:
Image('MindMapHypothesis.png') # ferramenta coggle


### 2.2 Criação de hipóteses 

#### 2.2.1 Hipóteses lojas

***1*** lojas com maior quadro de funcionarios deveriam vender mais

***2*** lojas com maior estoque deveriam vender mais

***3*** lojas com maior porte deveriam vender mais

***4*** lojas com maior sortimento deveriam vender mais

#### 2.2.2 Hipóteses Produtos

***1*** lojas que investem mais em marketing deveriam vender mais

***2*** lojas que expoem mais o produto deveriam vender mais

***3*** lojas que tem precos menores nos produtos deveriam vender mais

***4*** lojas que possuem precos menores por mais tempo deveriam vender mais


#### 2.2.3 Hipóteses Tempo

***1*** lojas que tem mais feriados deveriam vender mais

***2*** lojas que abrem nos primeiros 6 meses deveriam vender mais

***3*** lojas que abrem nos finais de semana deveriam vender mais

#### 2.2.4 Priorizaçao das hipóteses 

In [ ]:
# Aquelas que já possuimos os dados 

***1*** lojas com maior sortimento deveriam vender mais

***2*** lojas com competidores mais proximos deveriam vender menos

***3*** lojas com competidores a mais tempos deveriam vender mais

***4*** lojas com promocoes ativas por mais tempos deveriam vender mais

***5*** lojas com mais dias de promocao deveriam vender mais 

***6*** lojas com mais promocoes consecutivas deveriam vender mais

***7*** lojas abertas durante os feriados deveriam vender mais

***8*** lojas deveriam vender mais ao longo dos anos

***9*** lojas deveriam vender mais no segundo semestre

***10*** lojas deveriam vender mais depois do dia 10 de cada mes

***11*** lojas deveriam vender menos aos finais de semana

***12*** lojas deveriam vender menos durante feriados escolares



In [ ]:
df2=df1.copy()


### 2.3 Feature Engineering


In [ ]:
#desmembrando da varivel "date" com a funcao dt.year
df2['year']=df2['date'].dt.year   
df2['month']=df2['date'].dt.month
df2['day']=df2['date'].dt.day
df2['week_of_year']=df2['date'].dt.weekofyear
df2['year_week']=df2['date'].dt.strftime('%Y-%W') 

#juntar as duas variaveis - competition since
df2['competition_since']=df2.apply(lambda x:datetime.datetime(year=x['competition_open_since_year'],month=x['competition_open_since_month'],day=1),axis=1)
df2['competition_time_month']=((df2['date']- df2['competition_since'])/30).apply(lambda x:x.days).astype(int)

#promo since 
df2['promo_since']=df2['promo2_since_year'].astype(str)+'-'+ df2['promo2_since_week'].astype(str)
df2['promo_since']=df2['promo_since'].apply(lambda x:datetime.datetime.strptime(x +'-1','%Y-%W-%w')- datetime.timedelta(days=7))
df2['promo_time_week']=((df2['date']-df2['promo_since'])/7).apply(lambda x:x.days).astype(int)

##assortment 
df2['assortment']=df2['assortment'].apply(lambda x:'basic' if x=='a' else 'extra' if x=='b' else 'extended')

#
df2['state_holiday']= df2['state_holiday'].apply(lambda x:'public_holiday' if x=='a' else 'easter_holiday' if x=='b' else 'christmas' if x=='c'else 'regular_day')


In [ ]:
df2.head().T

# 3.0 Filtragem dos dados

In [ ]:
df3=df2.copy()
df3.head(15)

## 3.1 Filtragem de linhas 

In [ ]:
df3=df3[(df3['open']!=0)&(df3['sales']>0)]


## 3.2 Seleção de colunas 

In [ ]:
cols_drop=['customers','open','promo_interval','month_map']
df3=df3.drop(cols_drop,axis=1)
df3.columns

In [ ]:
df4=df3.copy()



# 4.0 Análise exploratória de dados 

## 4.1 Análise Univariada 

### 4.1.1 Response Variable

In [ ]:
plt.figure(figsize= (10, 7))
sns.distplot(df4['sales'], kde= True)
plt.tight_layout()

### 4.1.2 Numerical Variable 

In [ ]:
num_attributes.hist(bins=30,figsize=(15,15));

### 4.1.3 Categorical Variable

In [ ]:
cat_attributes.head()

In [ ]:
df4['state_holiday'].drop_duplicates()

In [ ]:
# state holiday
a=df4[df4['state_holiday']!= 'regular_day']
sns.countplot(a['state_holiday']);

In [ ]:
# state holiday + outros plots 
plt.figure(figsize= (10, 15))

plt.subplot(3,2,1)
a=df4[df4['state_holiday']!= 'regular_day']
sns.countplot(a['state_holiday'])

plt.subplot(3,2,2)
sns.kdeplot(df4[df4['state_holiday']=='public_holiday']['sales'],label='public_holiday',shade=True)
sns.kdeplot(df4[df4['state_holiday']=='easter_holiday']['sales'],label='easter_holiday',shade=True)
sns.kdeplot(df4[df4['state_holiday']=='christmas']['sales'],label='christmas',shade=True)


#store_type
plt.subplot(3,2,3)
sns.countplot(df4['store_type'])

plt.subplot(3,2,4)
sns.kdeplot(df4[df4['store_type']=='a']['sales'],label='a',shade=True)
sns.kdeplot(df4[df4['store_type']=='b']['sales'],label='b',shade=True)
sns.kdeplot(df4[df4['store_type']=='c']['sales'],label='c',shade=True)
sns.kdeplot(df4[df4['store_type']=='d']['sales'],label='d',shade=True)

# assortment
plt.subplot(3,2,5)
sns.countplot(df4['assortment'])

plt.subplot(3,2,6)
sns.kdeplot(df4[df4['assortment']=='extended']['sales'],label='extended',shade=True)
sns.kdeplot(df4[df4['assortment']=='basic']['sales'],label='basic',shade=True)
sns.kdeplot(df4[df4['assortment']=='extra']['sales'],label='extra',shade=True);
plt.tight_layout()


## 4.1 Análise Bivariada

***1*** lojas com maior sortimento deveriam vender mais

***2*** lojas com competidores mais proximos deveriam vender menos

***3*** lojas com competidores a mais tempos deveriam vender mais

***4*** lojas com promocoes ativas por mais tempos deveriam vender mais

***5*** lojas com mais dias de promocao deveriam vender mais 

***6*** lojas com mais promocoes consecutivas deveriam vender mais

***7*** lojas abertas durante os feriados deveriam vender mais

***8*** lojas deveriam vender mais ao longo dos anos

***9*** lojas deveriam vender mais no segundo semestre

***10*** lojas deveriam vender mais depois do dia 10 de cada mes

***11*** lojas deveriam vender menos aos finais de semana

***12*** lojas deveriam vender menos durante feriados escolares

## H1. Lojas com maior sortimento deveriam vender mais 



In [ ]:
aux1= df4[['assortment', 'sales']].groupby(['assortment']).sum().reset_index()
sns.barplot( x = 'assortment', y = 'sales', data = aux1);

aux2 = df4[['year_week', 'assortment', 'sales']].groupby(['year_week', 'assortment']).sum().reset_index()
aux2.pivot( index = 'year_week', columns = 'assortment', values = 'sales').plot();

aux3 = aux2[aux2['assortment'] == 'extra']
aux3.pivot( index = 'year_week', columns = 'assortment', values = 'sales').plot();

## H2. Lojas com competidores mais próximos  vendem menos.



In [ ]:
# agrupamento(bin) para melhor visualizacao de 1000 em 1000.

aux1=df4[['competition_distance','sales']].groupby('competition_distance').sum().reset_index()
plt.figure(figsize= (15, 10))


bins=list(np.arange(0,20000,1000)) #np.arrange= cria um array(lista de mesmo tipo)

aux1['competition_distance_binned']=pd.cut(aux1['competition_distance'],bins=bins) # cut= separa o valores da coluna 'comp_distance'no DataFrame df nas faixas etárias calculadas usando o valor do argumento bins no método pandas
aux2=aux1[['competition_distance_binned','sales']].groupby('competition_distance_binned').sum().reset_index()


sns.barplot(x='competition_distance_binned',y='sales',data=aux2);
plt.xticks(rotation=90);




In [ ]:
aux1=df4[['competition_distance','sales']].groupby('competition_distance').sum().reset_index()
plt.figure(figsize= (15, 15))

plt.subplot(1,3,1)
sns.scatterplot(x='competition_distance', y='sales',data=aux1);

plt.subplot(1,3,2)
bins=list(np.arange(0,20000,1000)) #np.arrange= cria um array(lista de mesmo tipo)
aux1['competition_distance_binned']=pd.cut(aux1['competition_distance'],bins=bins) # cut= separa o valores da coluna 'comp_distance'no DataFrame df nas faixas etárias calculadas usando o valor do argumento bins no método pandas
aux2=aux1[['competition_distance_binned','sales']].groupby('competition_distance_binned').sum().reset_index()
sns.barplot(x='competition_distance_binned',y='sales',data=aux2);
plt.xticks(rotation=90);


plt.subplot(1,3,3)
x=sns.heatmap(aux1.corr(method='pearson'),annot=True);
bottom,top=x.get_ylim()
x.set_ylim(bottom+0.5,top-0.5);


## H3.Lojas com competidores a mais tempo deveriam vender mais



In [ ]:
plt.figure(figsize= (15, 15))

plt.subplot(3, 1, 1)
aux1 = df4[['competition_time_month', 'sales']].groupby('competition_time_month').sum().reset_index()
aux2 = aux1[(aux1['competition_time_month'] < 120) & (aux1['competition_time_month'] != 0)]
sns.barplot(x = 'competition_time_month', y = 'sales', data = aux2);
plt.xticks( rotation = 90);

plt.subplot(3, 1, 2)
sns.regplot(x = 'competition_time_month', y = 'sales', data = aux2);

plt.subplot(3, 1, 3)
x = sns.heatmap(aux1.corr(method='pearson'), annot = True);
bottom, top = x.get_ylim()
x.set_ylim();

## H4. Lojas com promoçoes ativas por mais tempo vendem mais



In [ ]:
plt.figure(figsize= (15, 15))

plt.subplot(2,1,1)
sns.scatterplot(x= 'competition_time_month', y= 'sales', data= df4[(df4['competition_time_month'] <200) & df4['competition_time_month'] != 0])
titulo=('Tempo com Concorrência', 20);



In [ ]:
aux1=df4[['promo_time_week','sales']].groupby('promo_time_week').sum().reset_index()
plt.figure(figsize= (15, 15))

grid=GridSpec(2,3)

plt.subplot(grid[0,0])
aux2=aux1[aux1['promo_time_week']>0]
sns.barplot(x='promo_time_week',y='sales',data=aux2)
plt.xticks(rotation=90);


plt.subplot(grid[0,1])
sns.regplot(x='promo_time_week',y='sales',data=aux2)

plt.subplot(grid[1,0])
aux3=aux1[aux1['promo_time_week']<0]
sns.barplot(x='promo_time_week',y='sales',data=aux3)
plt.xticks(rotation=90);

plt.subplot(grid[1,1])
sns.regplot(x='promo_time_week',y='sales',data=aux3);

plt.subplot(grid[:,2])
sns.heatmap(aux1.corr(method='pearson'),annot=True);


##  H5. Lojas com mais promoçoes consecutivas vendem mais



In [ ]:
df4[['promo','promo2','sales']].groupby(['promo','promo2']).sum().reset_index()

In [ ]:
plt.figure(figsize= (15, 7))

aux1= df4[(df4['promo'] == 1) & (df4['promo2'] == 1)][['sales', 'year_week']].groupby('year_week').sum().reset_index()
sns.lineplot(x= 'year_week', y= 'sales', data= aux1, label= 'promo e promo2', estimator= sum)

aux2= df4[(df4['promo'] == 1) & (df4['promo2'] == 0)][['sales', 'year_week']].groupby('year_week').sum().reset_index()
sns.lineplot(x= 'year_week', y= 'sales', data= aux2, label= 'promo', estimator= sum)
plt.xticks(rotation= 90);

In [ ]:
# Lojas com promoções menos consecutivas vendem mais

aux1 = df4[(df4['promo'] == 1) & (df4['promo2'] == 1)][['year_week', 'sales']].groupby('year_week').sum().reset_index()
ax = aux1.plot()

aux2 = df4[(df4['promo'] == 1) & (df4['promo2'] == 0)][['year_week', 'sales']].groupby('year_week').sum().reset_index()
aux2.plot(ax=ax)

ax.legend(labels = ['Tradicional & Extendida', 'Extendida']);


## H6. Lojas que ficam abertas durante o feriado de Natal  vendem mais. 

In [ ]:
plt.figure(figsize= (15, 7))

plt.subplot(1,2,1)
aux=df4[df4['state_holiday']!='regular_day']
aux1=aux[['state_holiday','sales']].groupby('state_holiday').sum().reset_index()
sns.barplot(x='state_holiday',y='sales',data=aux1);


plt.subplot(1,2,2)
aux2=aux[['year', 'state_holiday','sales']].groupby(['year','state_holiday']).sum().reset_index()
sns.barplot(x='year',y='sales',hue='state_holiday',data=aux2);



## H7. Lojas devem vender mais ao longo dos anos.

In [ ]:
aux1= df4[['sales', 'year']].groupby('year').sum().reset_index()
plt.figure(figsize= (15, 7))

plt.subplot(1,3,1)
sns.barplot(x='year',y='sales', data=aux1);
plt.subplot(1,3,2)
sns.regplot(x='year',y='sales', data=aux1);

plt.subplot(1,3,3)
sns.heatmap(aux1.corr(method='pearson'), annot=True);


## H8. Lojas devem vender mais no segundo semestre do ano


In [ ]:
aux1= df4[['sales', 'month']].groupby('month').sum().reset_index()
plt.figure(figsize= (15, 7))

plt.subplot(1,3,1)
sns.barplot(x='month',y='sales', data=aux1);

plt.subplot(1,3,2)
sns.regplot(x='month',y='sales', data=aux1);

plt.subplot(1,3,3)
sns.heatmap(aux1.corr(method='pearson'), annot=True);

## H9. Lojas vendem mais depois do dia 10 de cada mês 


In [ ]:
aux1= df4[['sales', 'day']].groupby('day').sum().reset_index()
plt.figure(figsize= (15, 7))

plt.subplot(2,2,1)
sns.barplot(x='day',y='sales', data=aux1);

plt.subplot(2,2,2)
sns.regplot(x='day',y='sales', data=aux1);

plt.subplot(2,2,3)
sns.heatmap(aux1.corr(method='pearson'), annot=True);

aux1['before_after']= aux1['day'].apply(lambda x: 'before_10_days' if x<=10 else 'after_10_days')
aux2=aux1[['before_after','sales']].groupby ('before_after').sum().reset_index()

plt.subplot(2,2,4)
sns.barplot(x='before_after',y='sales', data=aux2);
plt.tight_layout()


In [ ]:
# gastos por dia do mes

aux1= df4[['sales', 'day']].groupby('day').sum().reset_index()
plt.figure(figsize= (15, 7))
sns.lineplot(x= 'day', y= 'sales', data= aux1, ci= None, estimator= sum)
plt.xticks(range(1,32));
plt.tight_layout()



## H10. Lojas deveriam vender menos no final de semana

In [ ]:
aux1= df4[['sales', 'day_of_week']].groupby('day_of_week').sum().reset_index()
plt.figure(figsize= (15, 7))

plt.subplot(2,2,1)
sns.barplot(x='day_of_week',y='sales', data= aux1);

plt.subplot(2,2,2)
sns.regplot(x='day_of_week',y='sales', data= aux1);

plt.subplot(2,2,3)
sns.heatmap(aux1.corr(method='pearson'), annot=True);



## H11. Lojas devem vender menos nos feriados escolares.


In [ ]:
plt.figure(figsize= (15, 7))

aux1= df4[['sales','school_holiday']].groupby('school_holiday').sum().reset_index()

plt.subplot(2,1,1)
sns.barplot(x='school_holiday', y='sales', data=aux1);

aux2= df4[['month','school_holiday','sales']].groupby(['month','school_holiday']).sum().reset_index()

plt.subplot(2,1,2)
sns.barplot(x='month',y='sales',hue='school_holiday', data=aux2);


In [ ]:
tab=[['Hipoteses','Conclusão','Relevância'],
     ['H1','Falsa','baixa'],
     ['H2','Falsa','média'],
     ['H3','Falsa','média'],
     ['H4','Falsa','baixa'],
     ['H5','Falsa','baixa'],
     ['H6','Falsa','baixa'],
     ['H7','Falsa','média'],
     ['H8','Falsa','alta'],
     ['H9','Falsa','alta'],
     ['H10','Verdadeira','alta'],
     ['H11','Verdadeira','alta'],
     ['H12','Verdadeira','baixa'],
    ]
print(tabulate(tab,headers='firstrow'))

## 4.3 Análise Multivariada 

## 4.3.1. Atributos Numéricos

In [ ]:
plt.figure(figsize= (15, 10))
correlation=num_attributes.corr(method='pearson')
sns.heatmap(correlation, annot= True, linewidths= 1);

# escuros: mais relacionados , claros: menos relacionados 

## 4.3.2. Atributos Categóricos

In [ ]:
df4.select_dtypes(include="object").head()

In [ ]:
# e necessario fazer a correlacao de variaveis categoricas- Nao e possivel fazer pelo metodo de pearson.
# E necessario usar o cramer -v

In [ ]:
#pd.crosstab(a['state_holiday'],a['store_type']) #possiveis combinacoes entre as variveis categoricas

In [ ]:
#ex:
#a[(a['state_holiday']=='christmas')&(a['store_type']=='a')]    
# o resultado 4 é a confusion matrix(cm)

In [ ]:
cm=pd.crosstab(a['state_holiday'],a['store_type'])

In [ ]:
chi2=ss.chi2_contingency(cm)

In [ ]:
chi2

In [ ]:
#(12792.159524019908) este e o valor de chi2
#ou
chi2=ss.chi2_contingency(cm)[0]
chi2

In [ ]:
cm

In [ ]:
#n: soma de todos os valores
n=cm.sum()
n

In [ ]:
# FORMULA v =np.sqrt((chi2/n)/(min(k-1,r-1)))

In [ ]:
a= df4.select_dtypes(include= 'object')

a1= cramer_v(a['state_holiday'], a['state_holiday'])
a2= cramer_v(a['state_holiday'], a['store_type'])
a3= cramer_v(a['state_holiday'], a['assortment'])
a4= cramer_v(a['store_type'], a['state_holiday'])
a5= cramer_v(a['store_type'], a['store_type'])
a6= cramer_v(a['store_type'], a['assortment'])
a7= cramer_v(a['assortment'], a['state_holiday'])
a8= cramer_v(a['assortment'], a['store_type'])
a9= cramer_v(a['assortment'], a['assortment'])

d= pd.DataFrame({'state_holiday': [a1, a2, a3],
                'store_type': [a4, a5, a6],
                'assortment': [a7, a8, a9]})

d= d.set_index(d.columns)
sns.heatmap(d, annot= True);

# 5.0 PREPARAÇÃO DOS DADOS


In [ ]:
df5= df4.copy()


## 5.1 Normalização


Como podemos observar na seção 4.1.2, não apresentamos nenhuma variável numérica com distribuição normal, então não faremos a normalização dos dados.
Rescala o centro para o 0 com desvio padrao igual a 1
= variavel - media/desvio padrao

## 5.2 Reescaling

Primeiro vamos selecionar as variáveis numéricas para depois observar as features que possuem outliers e as que não possuem para aplicar diferentes métodos de reescaling, sendo o RobustScaler para o primeiro e o MinMaxScaler para o segundo.
Rescala para o intervalo entre 0 e 1
Distribuicao nao Gaussianas

In [ ]:
df5.select_dtypes(include= ['int64', 'float64']).head()


In [ ]:
plt.figure(figsize= (15, 7))

lista= ['competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2_since_week','promo2_since_year', 'year']

n= 1
for i in lista:
    plt.subplot(3, 3, n)
    sns.boxplot(x= i, data= df5)
    plt.xlabel('')
    n+= 1

In [ ]:
rs= RobustScaler()  # quartis
mms= MinMaxScaler() 

#competition distance
df5['competition_distance'] = rs.fit_transform(df5[['competition_distance']].values)
pickle.dump(rs,open('parameter/encoding_competition_distance_scalerpkl','wb'))


#competition time month
df5['competition_time_month'] = rs.fit_transform(df5[['competition_time_month']].values)
pickle.dump(rs,open('parameter/encoding_competition_time_month_scaler.pkl','wb'))


#promo time week
df5['promo_time_week'] = mms.fit_transform(df5[['promo_time_week']].values)
pickle.dump(rs,open('parameter/encoding_promo_time_week_scaler.pkl','wb'))


# year
df5['year'] = mms.fit_transform(df5[['year']].values)

In [ ]:
fig=plt.figure(figsize=(9,8))
sns.distplot(df5['competition_distance'],bins= 30)
plt.tight_layout()


## 5.3 Transformação variáveis categóricas


### 5.3.1 Encoding

ENCODING = converter a variavel categorica em numerica mantendo o conteudo da variável

PRINCIPAIS TIPOS: one hot encoding, label, ordinal,target,frequency,embedding

LabelEncoding é uma classe de utilitário para ajudar a normalizar rótulos de forma que contenham apenas valores entre 0 e n_classes-1. Pode ser usado para transformar rótulos não numéricos em rótulos numéricos.

One Hot Encoding
Esse tipo de codificação pode ser obtido com o OneHotEncoder e get_dummies, que transforma cada recurso categórico com valores possíveis de n_categories em recursos binários de ncategories, sendo um deles 1 e todos os outros 0. Por padrão, os valores que cada recurso pode assumir são inferidos automaticamente do conjunto de dados e podem ser encontrados no atributo categorias.

Ordinal Encoding
Para converter recursos categóricos em códigos inteiros, podemos usar o OrdinalEncoder. Este estimador transforma cada recurso categórico em um novo recurso de números inteiros (0 a n_categories - 1) conforme ordenado.

Vamos pegar as features que são objects e transforma-las em numéricas através dos métodos abaixo:

state_holiday: OneHot Encoding
store_type: Label Enconding
assortment: Ordinal Encoding

In [ ]:
df5['assortment'].drop_duplicates()


In [ ]:
df5.head()

In [ ]:
# state_holiday - One Hot Encoding
df5 = pd.get_dummies(df5, prefix=['state_holiday'], columns = ['state_holiday'])


# store_type - Label Encoding
le = LabelEncoder()
df5['store_type'] = le.fit_transform(df5['store_type'])
pickle.dump(le,open ('parameter/store_type_scaler.pkl','wb'))

# assortment - Ordinal
assortment_dict = {'basic':1,
                  'extra': 2,
                  'extended':3}

df5['assortment'] = df5['assortment'].map(assortment_dict)


In [ ]:
# Após transformações
df5.head(15)

### 5.3.2 Response Variable Transformation


Objetivo de Todas as transformações: Tenta trazer distribuições que apresentam um skew mais a direita ou esquerda, mais próximo de uma normal.

logarithm Transformation -Aplicar o Log em todas as variáveis resposta.

Box-Cox Transformation - (Formula)

Cube-Root Transformation - Extrair a raiz cúbica de todos os valores.

Square-Root Transformation - Extrair a raiz quadrada de todos os valores.

Sine and Cosine Transformation - ... seno e cosseno.


### 5.3.3 Nature Transformation


In [ ]:
# nao se aplica no year pois nao se pode voltar no tempo, nao e ciclo é uma constante.

# month
df5['month_sin'] = df5['month'].apply(lambda x: np.sin(x * (2.*np.pi/12)))
df5['month_cos'] = df5['month'].apply(lambda x: np.cos(x * (2.*np.pi/12)))

# day
df5['day_sin'] = df5['day'].apply(lambda x: np.sin(x * (2.*np.pi/30)))
df5['day_cos'] = df5['day'].apply(lambda x: np.cos(x * (2.*np.pi/30)))

# week of year
df5['week_of_year_sin'] = df5['week_of_year'].apply(lambda x: np.sin(x * (2.*np.pi/52)))
df5['week_of_year_cos'] = df5['week_of_year'].apply(lambda x: np.cos(x * (2.*np.pi/52)))

# day of week
df5['day_of_week_sin'] = df5['day_of_week'].apply(lambda x: np.sin(x * (2.*np.pi/7)))
df5['day_of_week_cos'] = df5['day_of_week'].apply(lambda x: np.cos(x * (2.*np.pi/7)))

In [ ]:
df5.head()

### 5.3.3.1 Logarithm Transformation


In [ ]:
# Distribuição das vendas antes da transformação
plt.subplot(1,2,1)
sns.distplot(df5['sales'])

# aplicando a transformação Log
plt.subplot(1,2,2)
df5['sales_log'] = np.log1p(df5['sales'])
sns.distplot(df5['sales_log'])

### 5.3.3.2 Box-Cox Transformation


In [ ]:
# Distribuição das vendas antes da transformação
plt.subplot(1,2,1)
sns.distplot(df5['sales'])

# aplicando a transformação Box Cox
plt.subplot(1,2,2)
df5['sales_box'] = ss.boxcox(df5['sales'])[0]
sns.distplot(df5['sales_box'])


### 5.3.3.3 Square-Root Transformation


In [ ]:
# Distribuição das vendas antes da transformação
plt.subplot(1,2,1)
sns.distplot(df5['sales'])

# aplicando a transformação Square-Root
plt.subplot(1,2,2)
df5['sales_sqrt'] = np.sqrt(df5['sales'])
sns.distplot(df5['sales_sqrt'])

In [ ]:
# Compare
fig=plt.figure(figsize=(9,8))

sns.distplot(df5['sales_log'],label = 'log')
sns.distplot(df5['sales_sqrt'], label= 'sqrt')
sns.distplot(df5['sales_box'], label= 'box')
plt.legend()

In [ ]:
# usaremos
df5['sales'] = np.log1p(df5['sales'])

df5.head()

# 6.0. FEATURE SELECTION

In [ ]:
df6 = df5.copy()


Occam's Razor - A explicação mais simples sobre um fenômeno observado, deveria prevalecer sobre explicações mais complexas (Princípio da Navalha de Occam)

De modo geral, quanto mais simples for seu modelo, mais generalista ele será, quando mais informações (colunas), mais você especifica seu modelo para problemas específicos.

Tente sempre remover as variáveis colineares, que são variáveis que explicam a mesma coisa, pois lembre-se do princípio da navalha de Occam, quanto menos variáveis, mais simples, logo remova as v. colineares sempre.
Os 3 métodos de Seleção de Variáveis:
    
Seleção Univariada (Filter Methods)
vantagens:Método Simples, Método Rápido,Fácil Entendimento
desvantagens:Não considera a influência entre as variáveis (você corre o risco de excluir variáveis importantes quando combinadas com outras).
Seleção por Importância / Método Embutido (Embedded Methods)

Random Forest (classificação ou Regressão), Lasso Regression, Ridge Regression:
    
Seleção por Subset ( Wrapper Methods)
Ele testa as variáveis uma a uma e combinada, se aumentar a accuracy, ele deixa, se permanecer a mesma ou diminuir ele exclui. SELECIONE UMA VARIVAEL-> TREINE UM MODELO DE ML -> CALCULE A PERFORMANCE DO MODELO -> A PERFORMANCE AUMENTOU? MANTEM A VARIÁVEL ->ADICIONA UMA VARIVAEL ALEATORIA -> A VARIAVEL AUMENTO? SE NAO , REMOVE-SE A VARIAVEL -> ADICIONA OUTRA VARIAVEL E FAZ TODO O CICLO NOVAMENTE ATE A ACURACIA ESTAR SATISFATORIA .



## 6.1. Split Dataframe Into Training and Test Dataset

In [ ]:
df6.head()

In [ ]:
# deletar variaveis pois ja derivamos no item coseno e seno, estao repetidas.
# selecionar as variaveis com relacao ao tempo, de acordo com o solicitado no problema(6 semanas de previsao de faturamento)
cols_drop = ['week_of_year', 'day', 'month', 'day_of_week', 'promo_since', 'competition_since', 'year_week' ]
df6 = df6.drop( cols_drop, axis=1 )

In [ ]:
df6[['store', 'date']].groupby('store').max().reset_index()

In [ ]:
#agrupo por loja e venda   -
#metodo : - datetime.timedelta(days = 6*7= - 6 semanas antes do final , 6 semanas 7 dias 6*7
df6[['store', 'date']].groupby('store').min().reset_index()['date'][0] - datetime.timedelta(days = 6*7) 

In [ ]:
df6[['store', 'date']].groupby('store').max().reset_index()['date'][0] - datetime.timedelta(days = 6*7) 

In [ ]:
# selecionar a data das ultimas 6 semanas de vendas para teste

# training dataset
X_train = df6[df6['date'] < '2015-06-19']
y_train = X_train['sales']

# test dataset
X_test = df6[df6['date'] >= '2015-06-19']
y_test = X_test['sales']

print( 'Training Min Date: {}'.format( X_train['date'].min() ) )
print( 'Training Max Date: {}'.format( X_train['date'].max() ) )

print( '\nTest Min Date: {}'.format( X_test['date'].min() ) )
print( 'Test Max Date: {}'.format( X_test['date'].max() ) )


## 6.2. Boruta as Feature Selector


In [ ]:
#training and test dataset for Boruta
#X_train_n = X_train.drop( ['date', 'sales'], axis=1 ).values
#y_train_n = y_train.values.ravel() #ravel() transforma o resultado num vetor

## define RandomForestRegressor
#rf = RandomForestRegressor( n_jobs=-1 ) #padrao, roda melhor 

##define Boruta
#rf= random forest
# auto= a quantitadade de ramificacao da arvore
# verbise e random state= padrao(pode ser modificado)
#boruta = BorutaPy( rf, n_estimators='auto', verbose=2, random_state=42 ).fit( X_train_n, y_train_n )

### 6.2.1. Best Features from Boruta


In [ ]:
#cols_selected = boruta.support_.tolist()

## best features
#X_train_fs = X_train.drop( ['date', 'sales'], axis=1 )
#cols_selected_boruta = X_train_fs.iloc[:, cols_selected].columns.to_list()

## not selected boruta
#cols_not_selected_boruta = list( np.setdiff1d( X_train_fs.columns, cols_selected_boruta ) )

In [ ]:
# guardar o resultado do boruta para nao rodar o algoritmo novamente e travar a projeto.

cols_selected_boruta = [
    'store',
    'promo',
    'store_type',
    'assortment',
    'competition_distance',
    'competition_open_since_month',
    'competition_open_since_year',
    'promo2',
    'promo2_since_week',
    'promo2_since_year',
    'competition_time_month',
    'promo_time_week',
    'day_of_week_sin',
    'day_of_week_cos',
    'month_sin',
    'month_cos',
    'day_sin',
    'day_cos',
    'week_of_year_sin',
    'week_of_year_cos']

# columns to add
feat_to_add = ['date', 'sales']

cols_selected_boruta_full = cols_selected_boruta.copy()
cols_selected_boruta_full.extend(feat_to_add)

# 7.0. MACHINE LEARNING MODELS


Qual e o melhor modelo de ML para este projeto? Average model,linear regression,linear regression regularized,random forest regressor ou XGBost regressor?

Porque aplicar o ML? Aprender o comportamento de vendas com as varivaeis disponiveis e entao generalizar para o futuro


In [ ]:
x_train = X_train[cols_selected_boruta]
x_test = X_test[cols_selected_boruta]

# Time Series Data Preparation
x_training = X_train[cols_selected_boruta_full]

## 7.1 Average Model

In [ ]:
aux1 = x_test.copy() # cria copia para proteger dados originais
aux1['sales'] = y_test.copy()#

# prediction
aux2 = aux1[['store', 'sales']].groupby('store').mean().reset_index().rename(columns = {'sales': 'predictions'})
aux1 = pd.merge(aux1, aux2, how = 'left', on = 'store') # reescreveu aux1
yhat_baseline = aux1['predictions']  # yhat : estimado

# performance
baseline_result = ml_error('Average Model', np.expm1(y_test), np.expm1(yhat_baseline)) 
# ml error cria a funcao para todo o modelo ,#  ver 5.3.2 (log1p) - expm1 : variavel exponencial, volta ao varlor original antes do log1p
# ver funcoes 

baseline_result

## 7.2. Linear Regression Model

In [ ]:
# ver funcoes 
# model
lr = LinearRegression().fit(x_train, y_train) # .fit (): treina o modelo

# prediction
yhat_lr = lr.predict(x_test)

# performance
lr_result = ml_error('Linear Regression', np.expm1(y_test), np.expm1(yhat_lr))
lr_result

### 7.2.1 Linear Regression Model - Cross Validation

In [ ]:
lr_result_cv = cross_validation(x_training, 5 ,'Linear Regression', lr, verbose=True)
lr_result_cv

## 7.3 Linear Regression Lasso


In [ ]:
# model
lrr = Lasso(alpha = 0.01).fit(x_train, y_train) # alpha: o quanto vc quer de relevancia , que seja um valor comparavel

# prediction
yhat_lrr = lrr.predict(x_test)

# performance
lrr_result = ml_error('Linear Regression - Lasso', np.expm1(y_test), np.expm1(yhat_lrr))
lrr_result
# modelos lineares possuem pior resultado, contem mais erros.Necessario implementar modelos nao lineares


### 7.3.1 Linear Regression - Lasso - Cross Validation


In [ ]:
Lasso_cv = cross_validation(x_training,5,'Lasso', lrr, verbose=False)
Lasso_cv

## 7.4 Random Forest Regressor

In [ ]:
# model
rf = RandomForestRegressor(n_estimators = 60, n_jobs = -1, random_state = 42 ).fit(x_train, y_train)

# prediction
yhat_rf = rf.predict(x_test )

# performance
rf_result = ml_error( 'Random Forest Regressor', np.expm1( y_test ), np.expm1( yhat_rf ) )
rf_result

### 7.4.1 Random Forest Regressor - Cross Validation


In [ ]:
rf_cv = cross_validation(x_training,5,'Random Forest', rf)
rf_cv

## 7.5 XGBoost Regressor


In [ ]:
# model

model_xgb = xgb.XGBRegressor(objective = 'reg:squarederror',
                             n_estimators = 100, 
                             eta = 0.01,
                             max_depth = 10,
                             subsample = 0.7,
                             colsample_bytee = 0.9).fit(x_train, y_train)

# prediction#yhat_xgb = model_xgb.predict(x_test)

# performance
xgb_result = ml_error('XGBoost Regressor', np.expm1(y_test), np.expm1(yhat_xgb))
xgb_result

### 7.5.1 Xgboost - Cross Validation


In [ ]:
xbg_cv = cross_validation(x_training,5,'Xgboost', model_xgb)
xbg_cv

## 7.6. Compare Model's Performance


### 7.6.1. Single Performance


In [ ]:
modelling_result = pd.concat([baseline_result, lr_result, lrr_result, rf_result, xgb_result]  )
modelling_result

### 7.6.2. Real Performance - Cross Validation


In [ ]:
modelling_result_cv = pd.concat([lr_result_cv, Lasso_cv, rf_cv, xbg_cv]  )
modelling_result_cv

# 8.0 HYPERPARAMETER FINE TUNNING

RANDOM SEARCH : Define os valores para os parametros aleatoriamente (pros:mais rapido e mais facil, desvantagem: pode testar 2x os mesmos valores, pode nunca conseguir o melhor conjunto por ser aleatorio)

GRID SEARCH:Define todas as combinacoes possiveis de valores que os hiperparametros podem assumir(pros: vai encontrar o melhor, contra: demora muito tempo de ate meses)

BAYESIAN SEARCH: Define os valores para o s hiperparametros baseado na teoria de Bayes


## 8.1 Random Search


In [ ]:
param = {'n_estimators': [1500 ,1700 ,2500 ,2300 ,3500],  # original estimators 
        'eta': [0.01 , 0.03],
        'max_depth': [3, 5, 9],
        'subsample': [0.1, 0.5, 0.7],
        'colsample_bytree':[0.3, 0.7, 0.9],
        'min_child_weight':[3 ,8 ,15]
        }
MAX_EVAL = 10

In [ ]:
#final_result = pd.DataFrame()

for i in range(MAX_EVAL):
    Chosse values for parameter ramndomly
    hp = {k: random.sample(v,1)[0] for k,v in param.items() }
    print(hp)
    model
    model_xgb = xgb.XGBRegressor(objective = 'reg:squarederror',
                                  n_estimatiors =hp['n_estimators'],
                                 eta = hp['eta'],
                                 max_depth=hp['max_depth'],
                                 subsample =hp['subsample'],
                                 colsample_bytee = hp['colsample_bytree'],
                                 min_child_weight = hp['min_child_weight'])

    
    #performance
    result = cross_validation(x_training, 5 ,'XGBoost Regressor ', model_xgb, verbose = False )
    final_result = pd.concat([final_result, result])

final_result

## 8.2 Final Model


In [ ]:
#param_tuned = {'n_estimators':3000 ,
       'eta': 0.03,
       'max_depth':5,
       'subsample':0.7,
       'colsample_bytree':0.7,
       'min_child_weight':3
       }

In [ ]:
 #model final
model_xgb_tuned = xgb.XGBRegressor(objective = 'reg:squarederror',
                                 n_estimatiors =param_tuned['n_estimators'],
                                 eta = param_tuned['eta'],
                                 max_depth=param_tuned['max_depth'],
                                 subsample =param_tuned['subsample'],
                                 colsample_bytee = param_tuned['colsample_bytree'],
                                 min_child_weight = param_tuned['min_child_weight']).fit(x_train, y_train)

#predciton
yhat_xgb_tuned = model_xgb_tuned.predict(x_test)

#performance
xgb_result_tuned = ml_error('XGboost Regressor', np.expm1(y_test), np.expm1(yhat_xgb_tuned))
xgb_result_tuned

In [ ]:
#model
rf = RandomForestRegressor(n_estimators=300, n_jobs=-1,random_state=42).fit(x_train, y_train)

#prediction
yhat_rf = rf.predict(x_test)

#performance
rf_result = ml_error('Random Forest Regressor', np.expm1(y_test), np.expm1(yhat_rf) )
rf_result

# 9.0. Tradução e interpretação do erro


In [ ]:
df9 = X_test[cols_selected_boruta_full]

#Rescale
df9['sales'] = np.expm1(df9['sales'])
df9['predictions'] = np.expm1( yhat_rf )

## 9.1 Business Performance


In [ ]:
#Sum prediction
df91 = df9[['store', 'predictions']].groupby('store').sum().reset_index()

# MAE and MAPE
df9_aux1 = df9[['store','sales','predictions']].groupby('store').apply(lambda x: mean_absolute_error(x['sales'],x['predictions'])).reset_index().rename(columns = {0:'MAE'})
df9_aux2 = df9[['store','sales','predictions']].groupby('store').apply(lambda x: mean_absolute_percentage_error(x['sales'],x['predictions'])).reset_index().rename(columns = {0:'MAPE'})

#Merge
df9_aux3 = pd.merge(df9_aux1, df9_aux2, how='inner', on='store')
df92= pd.merge(df91, df9_aux3, how='inner', on='store')

#Scenarios
df92['worst_scenario'] = df92['predictions'] - df92['MAE']
df92['best_scenario'] = df92['predictions'] + df92['MAE']

#Order columns
df92 = df92[['store','predictions', 'worst_scenario','best_scenario','MAE','MAPE']]

In [ ]:
df92.sort_values('MAE', ascending = False).head()


In [ ]:
df92.sample(5)


In [ ]:
sns.scatterplot(x='store', y='MAPE', data=df92)


## 9.2 Total Performance


In [ ]:
df93 =df92[['predictions','worst_scenario','best_scenario']].apply(lambda x: np.sum(x), axis=0).reset_index().rename(columns={'index':'scenarios',0:'values'})
df93['values'] = df93['values'].map('R${:,.2f}'.format)
df93

## 9.3 Machine Learning Performance


In [ ]:
df9['error'] = df9['sales'] - df9['predictions']
df9['error_rate'] = df9['predictions'] / df9['sales']

In [ ]:
plt.subplot(2,2,1)
sns.lineplot(x='date', y='sales', data=df9, label='SALES')
sns.lineplot(x='date', y='predictions', data=df9, label='PREDICITONS')

plt.subplot(2,2,2)
sns.lineplot(x='date', y='error_rate', data= df9)
plt.axhline(1, linestyle = '--')

plt.subplot(2,2,3)
sns.distplot(df9['error'])

plt.subplot(2,2,4)
sns.scatterplot(df9['predictions'], df9['error'])

In [ ]:
pickle.dump(xgb_result_tuned, open ('parameters/model_xgb_rossman.pkl', 'wb'))
pickle.dump(rf_result, open ('parameters/model_rf_rossman.pkl', 'wb'))